In [43]:
import numpy as np
from collections import OrderedDict as odict
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

This notebook computes the latencies and bandwidths of the three primitive function types

In [44]:
#(hardware name, number of nodes)
filesD = {
         'knl_mpi1':('knl',1), 'knl_mpi2':('knl',2), 'knl_mpi4':('knl',4),
         'skl_mpi1':('skl',1), 'skl_mpi2':('skl',2), 'skl_mpi4':('skl',4),
         'i5':('i5',1),
         'p100_mpi1':('p100',1), 'p100_mpi2':('p100',2), 'p100_mpi4':('p100',4),
         'v100_mpi1':('v100',1), 'v100_mpi2':('v100',2), 'v100_mpi4':('v100',4),
         'gtx1060':('gtx1060',1)
        }

files = odict(sorted(filesD.items(), key= lambda t : t[1][1]))
pd.set_option('precision',1)

#### Axpby and Dot Latencies
The latencies are determined by taking the minimum of the average runtimes 

#### Axpby and Dot Bandwidths
The bandwidths are determined by taking the average bandwidth of the 30 bandwidths corresponding to the 3 largest sizes.

#### Dx-Dy Latencies
As in Axpby 

#### Dx-Dy Bandwidths
Since the efficiency of the matrix-vector multiplications depends on the polynomial coefficient we should compute these bandwidths separately


In [45]:
names={'axpby':3,'dot':2,'dx':3, 'dy':3}
#ns=[3,4]
values = []
for f, v in files.items() :#{'knl_mpi2':('knl',2)}.items():
    runtimes=pd.read_csv('benchmark_'+f+'.csv', delimiter=' ')
    #add size and bandwidth columns
    runtimes.insert(0,'size', 8*runtimes['n']*runtimes['n']
                    *runtimes['Nx']*runtimes['Ny']/1e6/v[1]) #inplace transformation
    for name,memops in names.items() :
        runtimes.insert(0,name+'_bw',runtimes['size']/1000*memops/runtimes[name])
    runtimes = runtimes.assign( dxdy=(runtimes['dx']+runtimes['dy'])/2)
    runtimes = runtimes.assign( dxdy_bw=2.0*runtimes['dx_bw']*runtimes['dy_bw']
                               /(runtimes['dx_bw']+runtimes['dy_bw']))
    #compute one version with aggregated grouped sizes and one without
    avgruntimes=runtimes.groupby(['n', 'Nx','Ny','size']).agg(['mean', 'std'])
    avgruntimes=avgruntimes.reset_index(level=['n','Nx','Ny','size'])
    avgruntimes.sort_values(by='size',inplace=True) #sort by size
    runtimes.sort_values(by='size',inplace=True)
    ##first compute axpby and dot latencies and bandwidths 
    nmax = 3
    s =30

    line = []
    l=len(runtimes)
    line.append(v[0]) #0
    line.append(v[1]) #1
    for q in ['axpby','dot']:
        bandwidth       = runtimes[l-s:l][q+'_bw'].mean()
        err_bandwidth   = runtimes[l-s:l][q+'_bw'].std()
        mean_latency    = avgruntimes[0:nmax][(q,'mean')].mean()/1e-6
        min_latency     = avgruntimes[(q,'mean')].min()/1e-6
        idx_min_latency = avgruntimes[(q,'mean')].idxmin()
        corr_min_latency= min_latency - avgruntimes['size'].loc[idx_min_latency]*names[q]/bandwidth/1e-3
        if corr_min_latency <0 : corr_min_latency = 0 
        line.append(bandwidth) #2 bandwidth
        line.append(err_bandwidth)  #3 err_bandwidth
        line.append(mean_latency) #4 latency mean
        line.append(min_latency)  #5 latency min
        line.append(corr_min_latency ) #6 corrected lat 

    ##now compute latency and bandwidths of dx and y
    for n in [2,3,4,5]:
        #take n
        dxdy=runtimes[runtimes['n']==n]
        
        avgdxdy = avgruntimes[avgruntimes['n']==n]
        dxdy=dxdy.sort_values(by='size')
        avgdxdy=avgdxdy.sort_values(by='size') #sort by size
        bandwidth       = dxdy[(dxdy['size']>10)&(dxdy['size']<1000)]['dxdy_bw'].mean()
        err_bandwidth   = dxdy[(dxdy['size']>10)&(dxdy['size']<1000)]['dxdy_bw'].std()
        mean_latency    = avgdxdy[0:nmax][('dxdy','mean')].mean()/1e-6
        min_latency     = avgdxdy[('dxdy','mean')].min()/1e-6
        idx_min_latency = avgdxdy[('dxdy','mean')].idxmin()
        corr_min_latency= min_latency - avgdxdy['size'].loc[idx_min_latency]*names['dx']/bandwidth/1e-3
        if corr_min_latency <0 : corr_min_latency = 0 
        line.append(bandwidth) #2 bandwidth
        line.append(err_bandwidth)  #3 err_bandwidth
        line.append(mean_latency) #4 latency mean
        line.append(min_latency)  #5 latency min
        line.append(corr_min_latency ) #6 corrected lat        
    #print(line)    
    values.append(line)

In [46]:
#now construct new table with values from previous cell      
tuples=[('arch','',''),('nodes','','')]        
for q in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append((q,'bw','avg'))
    tuples.append((q,'bw','std'))
    tuples.append((q,'lat','avg'))
    tuples.append((q,'lat','min'))
    tuples.append((q,'lat','bw'))

cols=pd.MultiIndex.from_tuples(tuples)
arr = pd.DataFrame(values,index=files.keys(), columns=cols)
arr.sort_values(by='arch',inplace=True)
arr.set_index(['arch','nodes'],inplace=True)
#arr.loc[:,[('dot','bw','avg'),('dot','lat','avg')]]
arr

axpby                              dot                         \
                  bw            lat                bw             lat          
                 avg      std   avg   min   bw    avg      std    avg    min   
arch    nodes                                                                  
gtx1060 1      157.0  5.6e-02  23.2   3.5  0.0   26.5  9.8e-02  199.9  131.6   
i5      1       30.0  1.9e-01  30.6  12.4  0.0    9.3  3.7e-02  316.7  117.4   
knl     1      437.6  3.3e+01  11.5   9.9  5.8  144.7  8.3e+00   75.3   62.0   
        2      372.4  2.3e+01  10.8  10.0  7.7  121.8  9.9e+00   99.3   92.8   
        4      424.4  4.5e+01  11.4  10.4  8.0  104.7  9.9e+00  132.0  128.3   
p100    1      552.8  1.0e+00   4.1   3.1  0.2  345.8  1.9e+00   61.1   56.4   
        2      554.2  1.1e-01   3.3   3.0  1.6  339.8  2.9e+00   53.5   49.6   
        4      554.7  8.9e-01   3.2   3.1  1.5  324.3  8.4e+00   54.0   48.7   
skl     1      207.5  3.2e+00   4.6   4.0  0.0  194.1  5.8e+00   32.3   23.1   
        2      216.4  4.8e+00  10.5   4.0  0.0  183.6  7.8e+00   54.5   25.3   
        4      233.9  9.8e+00   4.1   4.0  1.8  171.3  1.4e+01   38.2   37.1   
v100    1      845.6  9.2e-01   3.7   3.1  1.2  593.9  1.6e+00   36.6   35.7   
        2      846.9  4.3e-01   3.3   3.2  2.2  581.6  5.8e+00   38.3   36.0   
        4      844.8  1.8e+00   3.3   3.3  2.2  552.1  1.0e+01   40.0   37.4   

                      ...   dxdy4                             dxdy5         \
                      ...      bw           lat                  bw          
                  bw  ...     avg   std     avg    min    bw    avg    std   
arch    nodes         ...                                                    
gtx1060 1       92.1  ...    83.8  13.8   322.6   72.5   0.0   69.3   17.0   
i5      1        4.8  ...    25.6   1.5  1212.1  208.3   0.0   21.4    1.9   
knl     1       53.8  ...   130.8  14.4   212.4   48.0   0.0  104.8   12.9   
        2       88.0  ...   110.5  15.0   200.2   87.0  54.8   89.1    8.9   
        4      125.4  ...   118.5  17.5   139.5   73.4  58.4   91.3   14.9   
p100    1       53.4  ...   200.1   1.8   148.6   30.1   0.0  165.3   14.0   
        2       48.1  ...   184.7  13.0   128.1   66.6  49.6  158.9   11.3   
        4       47.9  ...   166.4  19.0   131.5   91.5  82.1  144.5   20.2   
skl     1       16.2  ...   117.5  21.2   390.1   85.9  18.2  112.3    8.7   
        2       21.7  ...   118.9  20.6   163.0   71.3  37.8  113.3    6.1   
        4       35.2  ...   114.6  19.5    97.6   54.7  37.4  109.8    6.7   
v100    1       34.0  ...   658.1  19.2    47.1   12.1   2.5  545.0   53.5   
        2       35.1  ...   539.1  87.8    72.6   52.9  47.1  472.9   81.7   
        4       36.3  ...   442.3  94.4   101.6   88.9  85.4  389.5  101.0   

                                    
                  lat               
                  avg    min    bw  
arch    nodes                       
gtx1060 1       571.4  125.0   0.0  
i5      1      1956.6  340.4   0.0  
knl     1       443.0   81.4   0.0  
        2       347.8  118.1  55.8  
        4       232.9  110.5  80.1  
p100    1       273.0   62.7   3.3  
        2       191.4   80.0  49.1  
        4       166.8  107.2  90.2  
skl     1       436.0  134.7  23.9  
        2       257.8   96.3  41.4  
        4       148.5   70.4  42.0  
v100    1        85.5   23.5   5.4  
        2        94.1   58.5  48.1  
        4       112.7   91.6  85.3  

[14 rows x 30 columns]

In [47]:
#arr=arr.reset_index()

In [48]:
#define conversion function 
def toString(x): 
    if pd.isnull(x) : return 'n/a'
    #string = '%.1f'% x
    string = '%d' %np.ceil(x)
    #if np.ceil(x)<100 : string = '0'+string
    if np.ceil(x)<10 : string = '0'+string
    return string

In [49]:
addto = []
for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    arr.loc[:,(n,'bw','string')]= arr[n]['bw']['avg'].apply(toString) +" ± "+arr[n]['bw']['std'].apply(toString)
    addto.append((n,'lat','bw'))
    addto.append((n,'bw','string'))

#make a table for display
nicetable=arr[addto]
drop = nicetable.columns.droplevel(2)
nicetable.columns=drop
#nicetable.reset_index(inplace=True)
#nicetable.set_index('arch')
newindex=[('i5',1)]
for n in ['skl','knl']:
    for m in [1,2,4]:
        newindex.append((n,m))
newindex.append(('gtx1060',1))
for n in ['p100','v100']:
    for m in [1,2,4]:
        newindex.append((n,m))
    
nicetable=nicetable.reindex(newindex)

nicetable

axpby              dot           dxdy2           dxdy3  \
                lat        bw    lat        bw   lat        bw   lat   
arch    nodes                                                          
i5      1       0.0   30 ± 01    4.8   10 ± 01   0.0   28 ± 03   0.0   
skl     1       0.0  208 ± 04   16.2  195 ± 06  27.0  182 ± 35  20.4   
        2       0.0  217 ± 05   21.7  184 ± 08  35.7  171 ± 45  36.2   
        4       1.8  234 ± 10   35.2  172 ± 14  37.5  163 ± 44  37.0   
knl     1       5.8  438 ± 34   53.8  145 ± 09   9.5  248 ± 26   7.9   
        2       7.7  373 ± 23   88.0  122 ± 10  45.1  179 ± 25  51.9   
        4       8.0  425 ± 46  125.4  105 ± 10  47.7  163 ± 14  54.9   
gtx1060 1       0.0  158 ± 01   92.1   27 ± 01   0.0  131 ± 01   2.7   
p100    1       0.2  553 ± 02   53.4  346 ± 02   1.9  288 ± 03   0.8   
        2       1.6  555 ± 01   48.1  340 ± 03  46.1  247 ± 21  45.5   
        4       1.5  555 ± 01   47.9  325 ± 09  82.6  206 ± 20  80.8   
v100    1       1.2  846 ± 01   34.0  594 ± 02   3.0  803 ± 17   2.9   
        2       2.2  847 ± 01   35.1  582 ± 06  49.4  574 ± 98  48.2   
        4       2.2  845 ± 02   36.3  553 ± 11  84.5  404 ± 61  83.9   

                         dxdy4           dxdy5             
                      bw   lat        bw   lat         bw  
arch    nodes                                              
i5      1        30 ± 03   0.0   26 ± 02   0.0    22 ± 02  
skl     1       162 ± 13  18.2  118 ± 22  23.9   113 ± 09  
        2       156 ± 15  37.8  119 ± 21  41.4   114 ± 07  
        4       152 ± 23  37.4  115 ± 20  42.0   110 ± 07  
knl     1       179 ± 24   0.0  131 ± 15   0.0   105 ± 13  
        2       145 ± 22  54.8  111 ± 16  55.8    90 ± 09  
        4       134 ± 26  58.4  119 ± 18  80.1    92 ± 15  
gtx1060 1       112 ± 02   0.0   84 ± 14   0.0    70 ± 18  
p100    1       238 ± 04   0.0  201 ± 02   3.3   166 ± 14  
        2       216 ± 15  49.6  185 ± 13  49.1   159 ± 12  
        4       177 ± 31  82.1  167 ± 19  90.2   145 ± 21  
v100    1       719 ± 23   2.5  659 ± 20   5.4   545 ± 54  
        2       580 ± 86  47.1  540 ± 88  48.1   473 ± 82  
        4      396 ± 127  85.4  443 ± 95  85.3  390 ± 102

#### Assumptions
- there are three basic functions: trivially parallel(axpby), nearest neighbor (dxdy), global reduction (dot)
- each can be represented by the single node bandwidth, the single node latency and the multinode latency

#### But
- does not capture cache effect e.g. in SKl

In [50]:
index = ['i5','skl','knl','gtx1060','p100','v100']  
lines = []
for arch in  index: 
    line = []
    line.append(arch)
    #first the bandwidths
    line.append( arr.loc[(arch,1),('axpby','bw','avg')] )
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        line.append( arr.loc[(arch,1),(n,'bw','avg')] /line[1])
    for n in ['axpby','dot','dxdy2'] :
        line.append( arr.loc[(arch,1),(n,'lat','bw')] )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(None)
        else:
            line.append( arr.loc[(arch,4),(n,'lat','bw')] )
    lines.append(line)
    
tuples=['arch']     

for n in ['axpby','dot','dxdy2','dxdy3','dxdy4','dxdy5']:
    tuples.append(n+'_bw')
for n in ['axpby','dot','dxdy']:
    tuples.append(n+'_lat_shared')
    tuples.append(n+'_lat_dist')
cols=tuples
toDisk = pd.DataFrame(lines, columns=cols)
toDisk.to_csv('performance.csv',sep=' ',index=False)

In [51]:
pd.set_option('precision',2)
test = pd.read_csv('performance.csv',delimiter=' ')
test

,arch,axpby_bw,dot_bw,dxdy2_bw,dxdy3_bw,dxdy4_bw,dxdy5_bw,axpby_lat_shared,axpby_lat_dist,dot_lat_shared,dot_lat_dist,dxdy_lat_shared,dxdy_lat_dist
0,i5,29.99,0.31,0.93,0.97,0.85,0.72,0.00,NaN,4.76,NaN,0.00,NaN
1,skl,207.52,0.94,0.87,0.78,0.57,0.54,0.00,1.84,16.25,35.16,26.99,37.55
2,knl,437.57,0.33,0.57,0.41,0.30,0.24,5.82,8.02,53.82,125.44,9.47,47.71
3,gtx1060,157.05,0.17,0.83,0.71,0.53,0.44,0.00,NaN,92.06,NaN,0.00,NaN
4,p100,552.83,0.63,0.52,0.43,0.36,0.30,0.25,1.53,53.35,47.86,1.91,82.57
5,v100,845.61,0.70,0.95,0.85,0.78,0.64,1.21,2.22,33.95,36.33,3.02,84.46


#### Observations
- note the high latency in the knl MPI implementation of dxdy. It seems to suffer from the same problem as the GPUs. (Is this the speed of PCIe we see?)

In [56]:
index = ['i5','skl','knl','gtx1060','p100','v100']  
#theo = [38,None,None,192,732,898]
lines = []
for arch in  index: 
    line = []
    #line.append(arch)
    #first the bandwidths
    base_bw = arr.loc[(arch,1), ('axpby','bw','avg')]
    err_bw = arr.loc[(arch,1), ('axpby','bw','std')]
    
    line.append( toString(base_bw)+" $\pm$ "+toString(err_bw) )
    line.append( toString(arr.loc[(arch,1),('axpby','lat','bw')]) )
    if arch == 'i5' or arch == 'gtx1060':
        line.append(toString(None))
    else:
        line.append( toString(arr.loc[(arch,4),('axpby','lat','bw')]) )
        
    for n in ['dot','dxdy2','dxdy3','dxdy4','dxdy5']:
        new_bw = arr.loc[(arch,1),(n,'bw','avg')]
        new_err = arr.loc[(arch,1),(n,'bw','std')]
        #efficiency
        #line.append( toString( new_bw/base_bw*100)+" $\pm$ "+toString(100*(err_bw/new_bw+new_bw/base_bw/base_bw*new_err))) 
        #bandwidth
        line.append( toString( new_bw)+" $\pm$ "+toString(new_err))
        line.append( toString(arr.loc[(arch,1),(n,'lat','bw')]) )
        if arch == 'i5' or arch == 'gtx1060':
            line.append(toString(None))
        else:
            if n == 'dot':
                line.append( toString(arr.loc[(arch,4),(n,'lat','bw')]) )
            else:
                line.append(toString(arr.loc[(arch,4),('dxdy2','lat','bw')]))
                
    lines.append(line)
    
tuples=[]  


for p in ['axpby','dot','dxdy (P=2)','dxdy (P=3)','dxdy (P=4)','dxdy (P=5)']:
    #for q in ['efficiency [\% bw]','lat s [us]','lat d [us]']:
    for q in ['bandwidth [GB/s]','lat 1 [us]','lat 4 [us]']:
        tuples.append((p,q))
tuples[0] = ('axpby','bandwidth [GB/s]')
    

cols=pd.MultiIndex.from_tuples(tuples)

toDisk = pd.DataFrame(lines, index=index, columns=cols)
#toDisk.insert(0,('theo','[GB/s]'),theo)
filename='axpby-dot.tex'
with open(filename, 'wb') as f:
    f.write(bytes(toDisk.iloc[:,0:6].to_latex(escape=False),'UTF-8'))
toDisk.iloc[:,0:6]

axpby                                    dot             \
        bandwidth [GB/s] lat 1 [us] lat 4 [us] bandwidth [GB/s] lat 1 [us]   
i5           30 $\pm$ 01         00        n/a      10 $\pm$ 01         05   
skl         208 $\pm$ 04         00         02     195 $\pm$ 06         17   
knl         438 $\pm$ 34         06         09     145 $\pm$ 09         54   
gtx1060     158 $\pm$ 01         00        n/a      27 $\pm$ 01         93   
p100        553 $\pm$ 02         01         02     346 $\pm$ 02         54   
v100        846 $\pm$ 01         02         03     594 $\pm$ 02         34   

                    
        lat 4 [us]  
i5             n/a  
skl             36  
knl            126  
gtx1060        n/a  
p100            48  
v100            37

In [57]:
dxdy = toDisk.loc[:,[('dxdy (P=2)','bandwidth [GB/s]'),
                     ('dxdy (P=3)','bandwidth [GB/s]'),
                     ('dxdy (P=4)','bandwidth [GB/s]'),
                     ('dxdy (P=5)','bandwidth [GB/s]'),
                     ('dxdy (P=2)','lat 1 [us]'),
                     ('dxdy (P=2)','lat 4 [us]'),
                    ]]
filename='dxdy.tex'
with open(filename, 'wb') as f:
    f.write(bytes(dxdy.to_latex(escape=False),'UTF-8'))
dxdy

dxdy (P=2)       dxdy (P=3)       dxdy (P=4)       dxdy (P=5)  \
        bandwidth [GB/s] bandwidth [GB/s] bandwidth [GB/s] bandwidth [GB/s]   
i5           28 $\pm$ 03      30 $\pm$ 03      26 $\pm$ 02      22 $\pm$ 02   
skl         182 $\pm$ 35     162 $\pm$ 13     118 $\pm$ 22     113 $\pm$ 09   
knl         248 $\pm$ 26     179 $\pm$ 24     131 $\pm$ 15     105 $\pm$ 13   
gtx1060     131 $\pm$ 01     112 $\pm$ 02      84 $\pm$ 14      70 $\pm$ 18   
p100        288 $\pm$ 03     238 $\pm$ 04     201 $\pm$ 02     166 $\pm$ 14   
v100        803 $\pm$ 17     719 $\pm$ 23     659 $\pm$ 20     545 $\pm$ 54   

        dxdy (P=2)             
        lat 1 [us] lat 4 [us]  
i5              00        n/a  
skl             27         38  
knl             10         48  
gtx1060         00        n/a  
p100            02         83  
v100            04         85